# 🧠 Study Buddy — Build Your Own RAG Chatbot with Gemini
Upload any PDF or text file (e.g., course notes, a Wikipedia export, or an article).

Ask questions like:
- “Summarize Chapter 2”
- “What is reinforcement learning?”
- “What’s the main takeaway from this section?”


In [1]:
# 🧩 Step 1: Install dependencies
!pip install -q google-generativeai PyPDF2 faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 50.1 MB/s eta 0:00:00


In [2]:
# 🧠 Step 2: Import libraries
import google.generativeai as genai
from getpass import getpass
import PyPDF2
import faiss
import numpy as np
import re

In [3]:
# ⚙️ Step 3: Configure Gemini API
GEMINI_API_KEY = getpass("🔑 Enter your Gemini API key: ")
genai.configure(api_key=GEMINI_API_KEY)

🔑 Enter your Gemini API key: ··········


In [4]:
# 🧾 Step 4: Upload your study material
from google.colab import files
uploaded = files.upload()

file_name = list(uploaded.keys())[0]
text = ""

if file_name.endswith(".pdf"):
    reader = PyPDF2.PdfReader(file_name)
    for page in reader.pages:
        text += page.extract_text() or ""
else:
    text = uploaded[file_name].decode("utf-8")

print(f"✅ Loaded {len(text)} characters from {file_name}")

Saving Risk Management in Trading Techniques to Drive Profitability of Hedge Funds and Trading Desks by Davis Edwards (z-lib.org) Chapter 8.pdf to Risk Management in Trading Techniques to Drive Profitability of Hedge Funds and Trading Desks by Davis Edwards (z-lib.org) Chapter 8.pdf
✅ Loaded 71396 characters from Risk Management in Trading Techniques to Drive Profitability of Hedge Funds and Trading Desks by Davis Edwards (z-lib.org) Chapter 8.pdf


In [5]:
# 🪄 Step 5: Split text into chunks
def split_text(text, chunk_size=1000, overlap=200):
    text = re.sub(r'\s+', ' ', text)
    chunks = []
    start = 0
    while start < len(text):
        end = start + chunk_size
        chunks.append(text[start:end])
        start += chunk_size - overlap
    return chunks

chunks = split_text(text)
print(f"📚 Split into {len(chunks)} chunks")

📚 Split into 88 chunks


In [6]:
# 🧩 Step 6: Create embeddings and index
embed_model = "models/gemini-embedding-001"
embeddings = []

for chunk in chunks:
    result = genai.embed_content(model=embed_model, content=chunk)
    embeddings.append(result["embedding"])

embeddings = np.array(embeddings, dtype="float32")

index = faiss.IndexFlatL2(embeddings.shape[1])
index.add(embeddings)
print("✅ Vector index built!")

✅ Vector index built!


In [12]:
# 💬 Step 7: Define RAG query function
def retrieve(query, k=3):
    q_embed = genai.embed_content(model=embed_model, content=query)["embedding"]
    _, idx = index.search(np.array([q_embed], dtype="float32"), k)
    return [chunks[i] for i in idx[0]]

def ask_study_buddy(query, k):
    docs = retrieve(query, k)
    context = "\n\n".join(docs)
    prompt = f"You are Study Buddy, a helpful assistant for learning.\nUse the context below to answer the question concisely and clearly.\n\nContext:\n{context}\n\nQuestion: {query}"
    model_name = "gemini-2.5-flash"
    model = genai.GenerativeModel(model_name)
    response = model.generate_content(prompt)
    return response.text

# 🧪 Step 8: Try asking a question
question = input("Enter your prompt: ")
k = input("Enter k (default 3): ")
print(f"🤔 Q: {question}\n")
print("💡 A:", ask_study_buddy(question, int(k)))

Enter your prompt: For each of the greeks (Delta, Gamma, Theta, Vega, Rho, Phi), write a one-sentence summary of what it represents. Then provide 1-2 examples of what different values of that greek might mean.
Enter k (default 3): 10
🤔 Q: For each of the greeks (Delta, Gamma, Theta, Vega, Rho, Phi), write a one-sentence summary of what it represents. Then provide 1-2 examples of what different values of that greek might mean.

💡 A: Here is a one-sentence summary and 1-2 examples for each Greek:

1.  **Delta:** Delta is a linear approximation that measures how much an option's value will change with respect to changes in the underlying asset's price.
    *   A higher Delta indicates the option price will move more quickly in response to a change in the underlying asset's price.
    *   A lower Delta suggests the option price will move less quickly compared to the underlying asset's price.

2.  **Gamma:** Gamma describes the curvature in an option's value and measures how much Delta chan

In [9]:
# For each of the greeks (Delta, Gamma, Theta, Vega, Rho, Phi), write a one-sentence summary of what it represents. Then provide 1-2 examples of what different values of that greek might mean.

# 🧪 Step 8: Try asking a question
question = input("Enter your prompt: ")
print(f"🤔 Q: {question}\n")
print("💡 A:", ask_study_buddy(question))

Enter your prompt: # For each of the greeks (Delta, Gamma, Theta, Vega, Rho, Phi), write a one-sentence summary of what it represents. Then provide 1-2 examples of what different values of that greek might mean.
🤔 Q: # For each of the greeks (Delta, Gamma, Theta, Vega, Rho, Phi), write a one-sentence summary of what it represents. Then provide 1-2 examples of what different values of that greek might mean.

💡 A: Here is a summary and examples for each Greek based on the provided context:

*   **Delta:** Delta is described as the first derivative of option price with respect to time. The provided context does not offer examples of what different values of Delta might mean.
*   **Gamma:** Gamma is described as the second derivative of option price with respect to time. The provided context does not offer examples of what different values of Gamma might mean.
*   **Theta:** Theta is not explicitly defined or mentioned in the provided context.
*   **Vega:** Vega is an estimate of how the o